In [254]:

import pathlib, glob
from datetime import datetime, timedelta
import pandas as pd

import tomllib as tomli


# === TOMLファイルの置き場所 =========================================
TOML_DIR = pathlib.Path(".")
PATTERN  = "*.toml"
# ===========================================================

def load_toml_meta(toml_dir: pathlib.Path, pattern: str = "*.toml") -> pd.DataFrame:
    """
    toml_dir にある TOML を走査し、ファイル名・start_date・lat・lon を
    DataFrame で返す。該当が無ければ空 DataFrame。
    """
    records = []
    for path in toml_dir.glob(pattern):
        with open(path, "rb") as f:
            d = tomli.load(f)

        # 位置と時刻の取得
        try:
            start_date_raw = d["observation_info"]["date_info"]["start_date"]
            lat, lon   = d["observation_info"]["location_info"]["position"]
        except KeyError as e:
            print(f"[warn] {path.name}: 必須キーが見つかりません → {e}")
            continue

        # 文字列 or datetime → datetime に統一
        if isinstance(start_date_raw, str):
            dt = datetime.fromisoformat(start_date_raw)
        else:
            dt = start_date_raw

        # JMA 表示用に調整：00:00:00 → 前日 24:00
        if dt.hour == 0 and dt.minute == 0 and dt.second == 0:
            jma_date = dt.date() - timedelta(days=1)
            jma_time = "24:00"
        else:
            jma_date = dt.date()
            jma_time = dt.strftime("%H:%M")

        records.append(
            {
                "file": path.name,
                "start_date": dt,
                "jma_date": jma_date,
                "jma_time": jma_time,
                "lat": lat,
                "lon": lon,
            }
        )

    if not records:
        print("⚠️  パターンに合う TOML が見つかりません。パスと PATTERN を確認してください。")
        return pd.DataFrame()

    df = pd.DataFrame(records).sort_values("file").reset_index(drop=True)
    return df


In [255]:
# ■ Cell 2 — アメダス観測所一覧を JSON でダウンロード
import requests

# 気象庁アメダス観測所一覧 JSON
URL_STATIONS = "https://www.jma.go.jp/bosai/amedas/const/amedastable.json"
with requests.get(URL_STATIONS) as resp:
    stations = resp.json()

# 辞書 → DataFrame に変換（行<->列を入れ替え）
stations_df = pd.DataFrame(stations).transpose()

In [256]:
# ■ Cell 3 — 観測所 DataFrame の前処理
import numpy as np

# (1) 緯度経度を [度, 分] → 十進度に変換
stations_df["lat"] = stations_df["lat"].apply(lambda x: x[0] + x[1]/60)
stations_df["lon"] = stations_df["lon"].apply(lambda x: x[0] + x[1]/60)

# (2) 観測要素 elems を分割して列を作る（降水量⇔precipitation, 風速⇔wind）
colnames = ["temperature","precipitation","windDirection","wind","sunshine","snowDepth","humidity","pressure"]
elems_arr = np.array(stations_df["elems"].apply(list).tolist()).astype(int)
for i, name in enumerate(colnames):
    stations_df[name] = elems_arr[:, i]

# (3) インデックスを station code（文字列）に
stations_df.index = stations_df.index.map(str)

stations_df.head()

,type,elems,lat,lon,alt,kjName,knName,enName,temperature,precipitation,windDirection,wind,sunshine,snowDepth,humidity,pressure
11001,C,11112010,45.520000,141.935000,26,宗谷岬,ソウヤミサキ,Cape Soya,1,1,1,1,2,0,1,0
11016,A,11111111,45.415000,141.678333,3,稚内,ワッカナイ,Wakkanai,1,1,1,1,1,1,1,1
11046,C,11112010,45.305000,141.045000,65,礼文,レブン,Rebun,1,1,1,1,2,0,1,0
11061,C,11110100,45.403333,141.801667,8,声問,コエトイ：稚内空港,Koetoi,1,1,1,1,0,1,0,0
11076,C,11112110,45.335000,142.170000,13,浜鬼志別,ハマオニシベツ,Hamaoni-Shibetsu,1,1,1,1,2,1,1,0


In [257]:
# ■ Cell — 最近傍のアメダス局を探す（meta_df の再定義付き）
import pandas as pd
from math import radians, cos, sin, asin, sqrt

# （1） まずは meta_df を再作成
meta_df = load_toml_meta(pathlib.Path("."), "*.toml")
meta_df["start_date"] = pd.to_datetime(meta_df["start_date"])

# （2） haversine と find_nearest_station を定義
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1; dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    return 2 * 6371 * asin(sqrt(a))

def find_nearest_station(lat, lon):
    dists = stations_df.apply(
        lambda row: haversine(lon, lat, row["lon"], row["lat"]), axis=1
    )
    return dists.idxmin()

# （3） 最近傍局を求めて列を追加
meta_df["amedas_id"] = meta_df.apply(
    lambda r: find_nearest_station(r.lat, r.lon), axis=1
)

meta_df.head()


,file,start_date,jma_date,jma_time,lat,lon,amedas_id
0,metadata_izuoshima.toml,2024-09-13,2024-09-12,24:00,34.413,139.022,44207


'24:00'

In [259]:
# ■ Cell — station_id から fuken_id をマッピングして 'prec' 列に保存
import pandas as pd
import pathlib

# (1) fuken.csv を読み込み（ヘッダーなし、列名を station_id, fuken_id に設定）
fuken_df = pd.read_csv(
    pathlib.Path("fuken.csv")
)
fuken_df["amedas_id"] = pd.to_numeric(fuken_df["amedas_id"], errors="coerce").astype("Int64")
fuken_df["station_id"] = pd.to_numeric(fuken_df["station_id"], errors="coerce").astype("Int64")
fuken_df["fuken_id"]    = pd.to_numeric(fuken_df["fuken_id"], errors="coerce").astype("Int64")

In [260]:
target_id = int(meta_df['amedas_id'][0])
exists = int(meta_df['amedas_id'][0]) in fuken_df["amedas_id"].values
if exists:
    match = fuken_df[fuken_df["amedas_id"] == target_id]
    print("該当行:")
    print(match)
else:
    print("該当する station_id は存在しません。")

該当行:
     amedas_id  station_id  fuken_id
525      44207          44      1522


In [261]:
# meta_df から日付を取得（0行目を例とする）
dt = meta_df.loc[0, "start_date"]  # または meta_df["start_date"][0]
block = int(match["fuken_id"].iloc[0])
prec = int(match["station_id"].iloc[0])

In [278]:
meta_df['jma_time'][0]

'24:00'

In [279]:
url = "https://www.data.jma.go.jp/stats/etrn/view/10min_a1.php?prec_no={}&block_no={:04}&year={}&month={}&day={}&view=".format(
    prec,
    block,
    dt.year,
    dt.month,
    dt.day
)

In [280]:
# ■ 2) ページを取得して pandas でテーブルを読み込む
resp = requests.get(url)
resp.raise_for_status()
# 先頭に現れる表を DataFrame 化

# ■ 3) StringIO に包んで読み込む
html_buf = io.StringIO(resp.text)
df_raw = pd.read_html(html_buf, header=[0])[0]

In [281]:
real_cols = df_raw.iloc[0].tolist()


In [282]:
# ■ 4) ４行目以降をデータ部として抜き出し、カラム名を real_cols に置き換え
dfAMEDAS = df_raw.iloc[3:].copy().reset_index(drop=True)
dfAMEDAS.columns = real_cols


In [283]:
dfAMEDAS = dfAMEDAS.rename(columns={
    real_cols[0]: "time",
    real_cols[1]: "precip_mm",
    real_cols[4]: "wind_avg_mps",
    real_cols[6]: "wind_gust_mps"
})

In [284]:
dfAMEDAS

,time,precip_mm,気温 (℃),相対湿度 (％),wind_avg_mps,wind_avg_mps,wind_gust_mps,wind_gust_mps,日照 時間 (min)
0,00:20,0.0,25.5,///,1.8,西南西,2.6,南西,///
1,00:30,0.0,25.3,///,1.7,西南西,2.6,西南西,///
2,00:40,0.0,25.4,///,1.5,西南西,1.5,南西,///
3,00:50,0.0,25.9,///,1.6,南西,2.6,南西,///
4,01:00,0.0,25.4,///,1.7,西南西,3.1,南西,///
...,...,...,...,...,...,...,...,...,...
138,23:20,0.0,27.4,///,1.7,西,2.6,西北西,///
139,23:30,0.0,27.2,///,1.4,西,2.1,西,///
140,23:40,0.0,27.1,///,1.8,西,2.6,西,///
141,23:50,0.0,26.8,///,1.9,西,2.6,西,///


In [290]:
jma_time = meta_df.loc[0, "jma_time"] 

In [294]:
# 1) 時刻文字列（例 "17:20"）を作成
time_str = dt.strftime("%H:%M")
jma_row = dfAMEDAS[dfAMEDAS["time"] == jma_time]

In [295]:
jma_row

,time,precip_mm,気温 (℃),相対湿度 (％),wind_avg_mps,wind_avg_mps,wind_gust_mps,wind_gust_mps,日照 時間 (min)
142,24:00,0.0,27.0,///,2.1,西,3.1,西南西,///
